In [1]:
import pandas as pd 
import numpy as np
import warnings
from jun_function import fmerge, kwd_len_mean, sess_dt_std
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Data Loading

In [11]:
target = pd.read_csv('cust_train.csv', encoding = 'UTF-8')
train = pd.read_csv('train.csv', encoding = 'UTF-8')
test = pd.read_csv('test.csv', encoding = 'UTF-8')
features = pd.read_csv('features.csv', encoding = 'UTF-8')
sparse = pd.read_csv('sparse_features.csv', encoding = 'UTF-8')

In [12]:
# 동일기준의 값으로 feature를 생성하기 위해서 train과 test를 concat
data = pd.concat([train, test])

In [13]:
# SESS_DT 열의 type을 datetime으로 바꾸기
data['SESS_DT'] = pd.to_datetime(data['SESS_DT'], format= '%Y%m%d')

# TOT_SESS_HR_V 열의 ,를 지우고 int로 변환
data['TOT_SESS_HR_V'] = data['TOT_SESS_HR_V'].apply(lambda x : int(x.replace(',','')))

# PD_BRA_NM 열의 불필요한 특수문자 제거
data.PD_BRA_NM = data.PD_BRA_NM.apply(lambda x : x.replace('[','').replace(']',''))

# Feature Making

In [14]:
f_list = []
s_list = []

## Product Features

**[총구매액, 구매건수, 평균구매액, 최대구매액, 최소구매액, 구매액표준편차, 구매액변동계수]** - (7)

In [6]:
f = data.groupby('CLNT_ID')['AMOUNT'].agg([('amount_sum', np.sum),
                                            ('buy_num', np.size),
                                            ('amount_mean', np.mean),
                                            ('amount_max', np.max),
                                            ('amount_min', np.min),
                                            ('amount_std', np.std),
                                            ('amount_cv', lambda x : np.std(x)/np.mean(x))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,amount_sum,buy_num,amount_mean,amount_max,amount_min,amount_std,amount_cv
0,0,86500,2,43250.000000,81000,5500,53386.561980,0.872832
1,1,1276000,16,79750.000000,99000,39000,22944.861444,0.278574
2,2,560000,11,50909.090909,62100,20000,13065.255799,0.244696
3,3,851200,8,106400.000000,110400,102400,4276.179871,0.037594
4,4,125200,6,20866.666667,37000,12800,12496.826264,0.546709
...,...,...,...,...,...,...,...,...
375859,375859,80000,3,26666.666667,35000,20000,7637.626158,0.233854
375860,375860,902800,12,75233.333333,149000,35400,38881.248889,0.494807
375861,375861,91800,2,45900.000000,45900,45900,0.000000,0.000000
375862,375862,92000,4,23000.000000,23000,23000,0.000000,0.000000


**[방문행동다양성]** - (1)

In [7]:
f = data.groupby('CLNT_ID')['HITS_SEQ'].agg([('visit_act_div','nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,visit_act_div
0,0,1
1,1,6
2,2,2
3,3,2
4,4,2
...,...,...
375859,375859,2
375860,375860,9
375861,375861,1
375862,375862,1


**[주방문행동]** - (500, 1)

In [15]:
fs = data.groupby(['CLNT_ID','HITS_SEQ'])['HITS_SEQ'].agg([
        ('hits_seq별 활동수',np.size)]).reset_index().sort_values(
        by = ['CLNT_ID','hits_seq별 활동수'], ascending = [True, False]).groupby('CLNT_ID').head(1)
fs.columns = ['CLNT_ID','most_visit_act','most_visit_act_cnt']
fs.most_visit_act = fs.most_visit_act.astype(str)
f = fs.iloc[:,[0,2]]
s = pd.get_dummies(fs.iloc[:,:2])
f_list.append(f); display(f)
s_list.append(s); display(s)

,CLNT_ID,most_visit_act_cnt
0,0,2
4,1,6
8,2,10
9,3,4
11,4,4
...,...,...
1147241,375859,2
1147247,375860,2
1147252,375861,2
1147253,375862,4


,CLNT_ID,most_visit_act_1,most_visit_act_10,most_visit_act_100,most_visit_act_101,most_visit_act_102,most_visit_act_103,most_visit_act_104,most_visit_act_105,most_visit_act_106,...,most_visit_act_90,most_visit_act_91,most_visit_act_92,most_visit_act_93,most_visit_act_94,most_visit_act_95,most_visit_act_96,most_visit_act_97,most_visit_act_98,most_visit_act_99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147241,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1147247,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1147252,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1147253,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[방문행동별 행동건수]** - (500)

In [7]:
s = pd.pivot_table(data, values = 'PD_C', index = 'CLNT_ID', columns = 'HITS_SEQ', 
                   aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

HITS_SEQ,CLNT_ID,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,500
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375860,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375862,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[여성브랜드의 구매액, 구매액비율, 구매건수, 구매건수비율]** - (4)

In [13]:
female_brand = ['입생로랑','레노마키즈','맥','나스','엠엘비키즈','난닝구','헤라','래핑차일드','에고이스트','랩']

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
fb = data.query('PD_BRA_NM == @female_brand').groupby('CLNT_ID')['AMOUNT'].agg([('female_brand_amount_sum','sum'),
                                                                                ('female_brand_buy_num','count')]).reset_index()
f = pd.merge(f, fb, how = 'left').fillna(0)
f['female_brand_amount_prob'] = f['female_brand_amount_sum']/f['총구매액']
f['female_brand_buy_prob'] = f['female_brand_buy_num']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,female_brand_amount_sum,female_brand_buy_num,female_brand_amount_prob,female_brand_buy_prob
0,0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0
...,...,...,...,...,...
375859,375859,0.0,0.0,0.0,0.0
375860,375860,902800.0,12.0,1.0,1.0
375861,375861,0.0,0.0,0.0,0.0
375862,375862,0.0,0.0,0.0,0.0


**[남성브랜드의 구매액, 구매액비율, 구매건수, 구매건수비율]** - (4)

In [15]:
mb_list = list(data.PD_BRA_NM.unique())
male_brand = [i for i in mb_list if '남성' in i]

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
mb = data.query('PD_BRA_NM == @male_brand').groupby('CLNT_ID')['AMOUNT'].agg([('male_brand_amount_sum','sum'),
                                                                              ('male_brand_buy_num','count')]).reset_index()
f = pd.merge(f, mb, how = 'left').fillna(0)
f['male_brand_amount_prob'] = f['male_brand_amount_sum']/f['총구매액']
f['male_brand_buy_prob'] = f['male_brand_buy_num']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,male_brand_amount_sum,male_brand_buy_num,male_brand_amount_prob,male_brand_buy_prob
0,0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0
...,...,...,...,...,...
375859,375859,0.0,0.0,0.0,0.0
375860,375860,0.0,0.0,0.0,0.0
375861,375861,0.0,0.0,0.0,0.0
375862,375862,0.0,0.0,0.0,0.0


**[화장품브랜드의 구매액, 구매액비율, 구매건수, 구매건수비율]** - (4)

In [16]:
makeup_brand_list = ["AGE20's",'AHC','APRILSKIN','DASHING DIVA','DHC','DMCK','HONG SHOT','LUK','Matricol','N.I.V lab',
                     'SECRET.Muse','SK-II','SPA THE EL HYAL STICK','VDL','VT 코스메틱','W피부연구소','가네보','가온도담','갸스비',
                     '게리쏭','겔랑','구달(화장품)','네이처리퍼블릭','니나리찌(화장품)','더 사가 오브 수(화장품)','더마토리(화장품)',
                     '동인비(화장품)','듀듀(화장품)','라곰(화장품)','라끄베르(화장품)','라베라(화장품)','라비오뜨(화장품)',
                     '로벡틴(화장품)','르샤트라1802','리파(명품화장품)','맥','메디큐브(화장품)','메디필(화장품)','모아트(화장품)',
                     '무(화장품)','미우미우(화장품)','미프(화장품)','바디판타지 (화장품)','바디판타지(화장품)','바이레도(화장품)',
                     '보닌(화장품)','보브(화장품)','비오템','비욘드(화장품)','세라마인(화장품)','셀퓨전씨(화장품)','셉(화장품)',
                     '소망화장품','수려한(화장품)','나스','입생로랑','베네피트','디올','헤라','조르지오 아르마니','메이크업포에버',
                     '에스티로더','슈에무라','어반디케이','크리니크','바비브라운','랑콤','키엘','빌리프','비오템','로라 메르시에',
                     '샤넬','아이오페','설화수','록시땅','알엠케이','이니스프리','마몽드','한율','에스쁘아']

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
mub = data.query('PD_BRA_NM == @makeup_brand_list').groupby('CLNT_ID')['AMOUNT'].agg([('makeup_brand_amount_sum','sum'),
                                                                                      ('makeup_brand_buy_num','count')]).reset_index()
f = pd.merge(f, mub, how = 'left').fillna(0)
f['makeup_brand_amount_prob'] = f['makeup_brand_amount_sum']/f['총구매액']
f['makeup_brand_buy_prob'] = f['makeup_brand_buy_num']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,makeup_brand_amount_sum,makeup_brand_buy_num,makeup_brand_amount_prob,makeup_brand_buy_prob
0,0,0.0,0.0,0.000000,0.000
1,1,728000.0,10.0,0.570533,0.625
2,2,0.0,0.0,0.000000,0.000
3,3,0.0,0.0,0.000000,0.000
4,4,0.0,0.0,0.000000,0.000
...,...,...,...,...,...
375859,375859,0.0,0.0,0.000000,0.000
375860,375860,0.0,0.0,0.000000,0.000
375861,375861,0.0,0.0,0.000000,0.000
375862,375862,0.0,0.0,0.000000,0.000


**[30대브랜드 구매액, 구매액비율, 구매건수, 구매건수비율]** - (4)

In [17]:
thirty_brand = ['래핑차일드','오가닉맘','해피프린스','밍크뮤']

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
thb = data.query('PD_BRA_NM == @thirty_brand').groupby('CLNT_ID')['AMOUNT'].agg([('thirty_brand_amount_sum','sum'),
                                                                                 ('thirty_brand_buy_num','count')]).reset_index()
f = pd.merge(f, thb, how = 'left').fillna(0)
f['thirty_brand_amount_prob'] = f['thirty_brand_amount_sum']/f['총구매액']
f['thirty_brand_buy_prob'] = f['thirty_brand_buy_num']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,thirty_brand_amount_sum,thirty_brand_buy_num,thirty_brand_amount_prob,thirty_brand_buy_prob
0,0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0
...,...,...,...,...,...
375859,375859,0.0,0.0,0.0,0.0
375860,375860,0.0,0.0,0.0,0.0
375861,375861,0.0,0.0,0.0,0.0
375862,375862,0.0,0.0,0.0,0.0


**[NOT20대브랜드 구매액, 구매액비율, 구매건수, 구매건수비율]** - (4)

In [20]:
not_twenty_brand = ['블루독','에이치커넥트','첨이첨이','레노마키즈','엠엘비키즈',
                    '정관장','노스페이스키즈(아동)','무냐무냐','난닝구','바보사랑']

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
ntwb = data.query('PD_BRA_NM == @not_twenty_brand').groupby('CLNT_ID')['AMOUNT'].agg([('not_twenty_brand_amount_sum','sum'),
                                                                                    ('not_twenty_brand_buy_num','count')]).reset_index()
f = pd.merge(f, ntwb, how = 'left').fillna(0)
f['not_twenty_brand_amount_prob'] = f['not_twenty_brand_amount_sum']/f['총구매액']
f['not_twenty_brand_buy_prob'] = f['not_twenty_brand_buy_num']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,not_twenty_brand_amount_sum,not_twenty_brand_buy_num,not_twenty_brand_amount_prob,not_twenty_brand_buy_prob
0,0,0.0,0.0,0.00000,0.000
1,1,192000.0,2.0,0.15047,0.125
2,2,0.0,0.0,0.00000,0.000
3,3,0.0,0.0,0.00000,0.000
4,4,0.0,0.0,0.00000,0.000
...,...,...,...,...,...
375859,375859,0.0,0.0,0.00000,0.000
375860,375860,0.0,0.0,0.00000,0.000
375861,375861,0.0,0.0,0.00000,0.000
375862,375862,0.0,0.0,0.00000,0.000


**[구매브랜드다양성]** - (1)

In [22]:
f = data.groupby('CLNT_ID')['PD_BRA_NM'].agg([('buy_brand_num', 'nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,buy_brand_num
0,0,2
1,1,5
2,2,3
3,3,2
4,4,2
...,...,...
375859,375859,1
375860,375860,1
375861,375861,1
375862,375862,1


**[최다구매브랜드구매건수]** - (1)

In [7]:
f = data.groupby(['CLNT_ID','PD_BRA_NM'])['PD_BRA_NM'].agg([
        ('브랜드별구매건수',np.size)]).reset_index().sort_values(
        by = ['CLNT_ID','브랜드별구매건수'], ascending = [True, False]).groupby('CLNT_ID').head(1)
f.columns = ['CLNT_ID','most_buy_brand','most_buy_brand_cnt']
f = fs.iloc[:,[0,2]]
f_list.append(f); display(f)

,CLNT_ID,most_buy_brand_cnt
0,0,1
6,1,9
8,2,5
10,3,4
13,4,4
...,...,...
1232552,375859,3
1232553,375860,12
1232554,375861,2
1232555,375862,4


,CLNT_ID,most_buy_brand_#텐텐,most_buy_brand_(유)자연식품,most_buy_brand_(주)다경,most_buy_brand_(주)두문,most_buy_brand_(주)두잇,most_buy_brand_(주)비엘퓨리티,most_buy_brand_1+1,most_buy_brand_100%국내생산,most_buy_brand_100090,...,most_buy_brand_히키스,most_buy_brand_히타치,most_buy_brand_힉스,most_buy_brand_힐링(식품),most_buy_brand_힐링버드,most_buy_brand_힐링쉴드,most_buy_brand_힐링타임,most_buy_brand_힐스,most_buy_brand_힙스터키드,most_buy_brand_힙핑크
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232552,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1232553,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1232554,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1232555,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[고,저가상품 구매액, 구매액비율, 구매건수, 구매건수비율]** - (8)

In [24]:
high_low_4q = list(data.PD_BUY_AM.describe().reset_index().iloc[4:7,1])

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()

h = data.query('PD_BUY_AM > @high_low_4q[2]').groupby('CLNT_ID')['AMOUNT'].agg([('high_pd_amount_sum','sum'),
                                                                                ('high_pd_buy_num','count')]).reset_index()
l = data.query('PD_BUY_AM < @high_low_4q[0]').groupby('CLNT_ID')['AMOUNT'].agg([('low_pd_amount_sum','sum'),
                                                                                ('low_pd_buy_num','count')]).reset_index()
f = pd.merge(f, h, how = 'left').fillna(0)
f = pd.merge(f, l, how = 'left').fillna(0)
f['high_pd_amount_prob'] = f['high_pd_amount_sum']/f['총구매액']
f['low_pd_amount_prob'] = f['low_pd_amount_sum']/f['총구매액']
f['high_pd_buy_prob'] = f['high_pd_buy_num']/f['구매건수']
f['low_pd_buy_prob'] = f['low_pd_buy_num']/f['구매건수']
f = f.iloc[:,[0,3,4,5,6,7,8,9,10]]
f_list.append(f); display(f)

,CLNT_ID,high_pd_amount_sum,high_pd_buy_num,low_pd_amount_sum,low_pd_buy_num,high_pd_amount_prob,low_pd_amount_prob,high_pd_buy_prob,low_pd_buy_prob
0,0,81000.0,1.0,5500.0,1.0,0.936416,0.063584,0.500000,0.500000
1,1,1237000.0,15.0,0.0,0.0,0.969436,0.000000,0.937500,0.000000
2,2,310500.0,5.0,20000.0,1.0,0.554464,0.035714,0.454545,0.090909
3,3,851200.0,8.0,0.0,0.0,1.000000,0.000000,1.000000,0.000000
4,4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
375859,375859,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
375860,375860,789400.0,9.0,0.0,0.0,0.874391,0.000000,0.750000,0.000000
375861,375861,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
375862,375862,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


**[총구매물건수, 구매물건수규칙성, 평균구매물건수]** - (3)

In [25]:
f = train.groupby('CLNT_ID')['PD_BUY_CT'].agg([('buy_pd_cnt_sum',np.sum),
                                               ('buy_pd_cnt_std',np.std),
                                               ('buy_pd_cnt_mean',np.mean)]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,buy_pd_cnt_sum,buy_pd_cnt_std,buy_pd_cnt_mean
0,0,2,0.000000,1.000000
1,1,16,0.000000,1.000000
2,2,12,0.301511,1.090909
3,3,8,0.000000,1.000000
4,4,6,0.000000,1.000000
...,...,...,...,...
263099,263099,112,0.000000,1.000000
263100,263100,2,0.000000,1.000000
263101,263101,4,0.000000,1.000000
263102,263102,55,3.903600,3.666667


## Master Features

**[유아물품구매건수]** - (4)

In [26]:
baby_pd_list = list(train.CLAC2_NM.unique())
baby_pd_list = [i for i in baby_pd_list if '유아' in i or '남아' in i or '여아' in i]

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
bpd = data.query('CLAC2_NM == @baby_pd_list').groupby('CLNT_ID')['AMOUNT'].agg([('baby_pd_amount_sum','sum'),
                                                                                ('baby_pd_buy_num','count')]).reset_index()
f = pd.merge(f, bpd, how = 'left').fillna(0)
f['baby_pd_amount_prob'] = f['baby_pd_amount_sum']/f['총구매액']
f['baby_pd_buy_prob'] = f['baby_pd_buy_num']/f['구매건수']

f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,baby_pd_amount_sum,baby_pd_buy_num,baby_pd_amount_prob,baby_pd_buy_prob
0,0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0
2,2,560000.0,11.0,1.0,1.0
3,3,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0
...,...,...,...,...,...
375859,375859,0.0,0.0,0.0,0.0
375860,375860,0.0,0.0,0.0,0.0
375861,375861,0.0,0.0,0.0,0.0
375862,375862,0.0,0.0,0.0,0.0


**[여성물품 구매건수]** - (4)

In [27]:
female_pd_list = list(train.CLAC2_NM.unique())
female_pd_list = [i for i in female_pd_list if '여성' in i]

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
fpd = data.query('CLAC2_NM == @female_pd_list').groupby('CLNT_ID')['AMOUNT'].agg([('female_pd_amount_sum','sum'),
                                                                                   ('female_pd_buy_num','count')]).reset_index()
f = pd.merge(f, fpd, how = 'left').fillna(0)
f['female_pd_amount_prob'] = f['female_pd_amount_sum']/f['총구매액']
f['female_pd_buy_prob'] = f['female_pd_buy_num']/f['구매건수']

f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,female_pd_amount_sum,female_pd_buy_num,female_pd_amount_prob,female_pd_buy_prob
0,0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0
3,3,851200.0,8.0,1.0,1.0
4,4,0.0,0.0,0.0,0.0
...,...,...,...,...,...
375859,375859,0.0,0.0,0.0,0.0
375860,375860,902800.0,12.0,1.0,1.0
375861,375861,0.0,0.0,0.0,0.0
375862,375862,0.0,0.0,0.0,0.0


**[남성물품 구매건수]** - (4)

In [28]:
male_pd_list = list(train.CLAC2_NM.unique())
male_pd_list = [i for i in male_pd_list if '남성' in i]

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액',np.sum),('구매건수',np.size)]).reset_index()
mpd = data.query('CLAC2_NM == @male_pd_list').groupby('CLNT_ID')['AMOUNT'].agg([('male_pd_amount_sum','sum'),
                                                                                ('male_pd_buy_num','count')]).reset_index()
f = pd.merge(f, mpd, how = 'left').fillna(0)
f['male_pd_amount_prob'] = f['male_pd_amount_sum']/f['총구매액']
f['male_pd_buy_prob'] = f['male_pd_buy_num']/f['구매건수']

f = f.iloc[:,[0,3,4,5,6]]
f_list.append(f); display(f)

,CLNT_ID,male_pd_amount_sum,male_pd_buy_num,male_pd_amount_prob,male_pd_buy_prob
0,0,0.0,0.0,0.000000,0.000000
1,1,0.0,0.0,0.000000,0.000000
2,2,0.0,0.0,0.000000,0.000000
3,3,0.0,0.0,0.000000,0.000000
4,4,51200.0,4.0,0.408946,0.666667
...,...,...,...,...,...
375859,375859,80000.0,3.0,1.000000,1.000000
375860,375860,0.0,0.0,0.000000,0.000000
375861,375861,0.0,0.0,0.000000,0.000000
375862,375862,92000.0,4.0,1.000000,1.000000


**[상품분류별 구매건수 - 대,중,소]** - (37, 128, 893)

In [8]:
s = pd.pivot_table(data, values = 'PD_C', index = 'CLNT_ID', columns = 'CLAC1_NM',
              aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

s = pd.pivot_table(data, values = 'PD_C', index = 'CLNT_ID', columns = 'CLAC2_NM',
              aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

s = pd.pivot_table(data, values = 'PD_C', index = 'CLNT_ID', columns = 'CLAC3_NM',
              aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

CLAC1_NM,CLNT_ID,가구,건강식품,계절가전,과일,구기/필드스포츠,남성의류,냉동식품,냉장/세탁가전,냉장식품,...,주방잡화,청소/세탁/욕실용품,축산물,출산/육아용품,침구/수예,컴퓨터,패션잡화,퍼스널케어,헬스/피트니스,화장품/뷰티케어
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8,0,0,0
4,4,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
375860,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375862,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


CLAC2_NM,CLNT_ID,TV,거실가구,건강보조식품,건강진액,견과류,고양이용품,골프,공기청정/가습/제습,교육완구,...,패션액세서리,포장반찬,피트니스,필기도구,핸드/풋케어,향수,헤어케어,홈웨어,홍삼/인삼가공식품,화장지/티슈
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375860,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375862,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


CLAC3_NM,CLNT_ID,2단우산,3단우산,BB/파운데이션/컴팩트류,DIY가구,DSLR,LCD,LED,OLED,PC부품,...,홈웨어세트,홍삼/인삼혼합세트,홍삼근,홍삼액,홍삼절편,홍삼정/분말/환,화장대,환풍기,휴대폰,힙색/사이드백
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375860,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375862,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**[구매상품다양성 - 대, 중, 소]** - (3)

In [30]:
# 구매상품다양성 - 대
f = data.groupby('CLNT_ID')['CLAC1_NM'].agg([('pur_pd_div_1', 'nunique')]).reset_index()
f_list.append(f); display(f)

# 구매상품다양성 - 중
f = data.groupby('CLNT_ID')['CLAC2_NM'].agg([('pur_pd_div_2', 'nunique')]).reset_index()
f_list.append(f); display(f)

# 구매상품다양성 - 소
f = data.groupby('CLNT_ID')['CLAC3_NM'].agg([('pur_pd_div_3', 'nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,pur_pd_div_1
0,0,2
1,1,3
2,2,2
3,3,1
4,4,2
...,...,...
375859,375859,1
375860,375860,1
375861,375861,1
375862,375862,1


,CLNT_ID,pur_pd_div_2
0,0,2
1,1,5
2,2,2
3,3,1
4,4,2
...,...,...
375859,375859,1
375860,375860,2
375861,375861,1
375862,375862,1


,CLNT_ID,pur_pd_div_3
0,0,2
1,1,7
2,2,3
3,3,1
4,4,2
...,...,...
375859,375859,1
375860,375860,5
375861,375861,1
375862,375862,1


**[최다구매상품분류 - 대,중,소]** - (37, 128, 872)

In [ ]:
s = data.groupby('CLNT_ID')['CLAC1_NM'].agg([('most_buy_clac1', lambda x : x.value_counts().index[0])]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

s = data.groupby('CLNT_ID')['CLAC2_NM'].agg([('most_buy_clac2', lambda x : x.value_counts().index[0])]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

s = data.groupby('CLNT_ID')['CLAC3_NM'].agg([('most_buy_clac3', lambda x : x.value_counts().index[0])]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

## Search Features

**[최다검색키워드검색건수]** - (1)

In [63]:
f = data.groupby(['CLNT_ID','KWD_NM'])['SEARCH_CNT'].agg([
    ('키워드별검색건수',lambda x : sum(x))]).reset_index().sort_values(
    by = ['CLNT_ID','키워드별검색건수'],ascending = [True, False]).groupby('CLNT_ID').head(1)
f.columns = ['CLNT_ID','most_search_kwd','most_search_kwd_cnt']
f = f.iloc[:,[0,2]]
f_list.append(f); display(f)

,CLNT_ID,most_search_kwd_cnt
0,0,2
7,1,6
15,2,6
16,3,6
20,4,4
...,...,...
2129183,375859,9
2129184,375860,14
2129187,375861,2
2129189,375862,2


**[검색키워드수, 검색키워드평균길이]** - (2)

In [9]:
f = data.groupby('CLNT_ID')['KWD_NM'].agg([('search_kwd_sum', 'nunique'),
                                           ('kwd_len_mean', lambda x : kwd_len_mean(x))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,search_kwd_sum,kwd_len_mean
0,0,1,6.000000
1,1,9,4.375000
2,2,6,5.818182
3,3,4,4.500000
4,4,4,7.166667
...,...,...,...
375859,375859,1,3.000000
375860,375860,3,6.416667
375861,375861,2,8.000000
375862,375862,2,6.500000


**[총검색건수, 평균검색건수, 최대검색건수]** - (3)

In [10]:
f = data.groupby('CLNT_ID')['SEARCH_CNT'].agg([('search_cnt_sum','sum'),
                                               ('search_cnt_mean','mean'),
                                               ('search_cnt_max','max')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,search_cnt_sum,search_cnt_mean,search_cnt_max
0,0,2,1.000000,1
1,1,23,1.437500,2
2,2,23,2.090909,3
3,3,12,1.500000,3
4,4,11,1.833333,2
...,...,...,...,...
375859,375859,9,3.000000,3
375860,375860,23,1.916667,5
375861,375861,3,1.500000,2
375862,375862,4,1.000000,1


## Session Features

**[총방문세션]** - (1)

In [11]:
f = data.groupby('CLNT_ID')['SESS_ID'].agg([('visit_sess_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,visit_sess_cnt
0,0,1
1,1,5
2,2,2
3,3,1
4,4,2
...,...,...
375859,375859,1
375860,375860,9
375861,375861,1
375862,375862,1


**[총방문일수]** - (1)

In [12]:
f = data.groupby('CLNT_ID')['SESS_DT'].agg([('visit_dt_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,visit_dt_cnt
0,0,1
1,1,4
2,2,2
3,3,1
4,4,2
...,...,...
375859,375859,1
375860,375860,9
375861,375861,1
375862,375862,1


**[요일별방문비율]** - (7)

In [14]:
f = data.groupby('CLNT_ID')['SESS_DT'].agg([('mon_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([0]))),
                                            ('tue_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([1]))),
                                            ('wen_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([2]))),
                                            ('thu_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([3]))),
                                            ('fri_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([4]))),
                                            ('sat_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([5]))),
                                            ('sun_visit_prob', lambda x : np.mean(x.dt.dayofweek.isin([6])))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,mon_visit_prob,tue_visit_prob,wen_visit_prob,thu_visit_prob,fri_visit_prob,sat_visit_prob,sun_visit_prob
0,0,0.000000,0.000000,0.0000,0.000000,0.000000,1.000000,0.000000
1,1,0.000000,0.687500,0.1875,0.000000,0.000000,0.125000,0.000000
2,2,0.090909,0.000000,0.0000,0.909091,0.000000,0.000000,0.000000
3,3,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1.000000
4,4,0.000000,0.666667,0.0000,0.000000,0.000000,0.333333,0.000000
...,...,...,...,...,...,...,...,...
375859,375859,0.000000,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000
375860,375860,0.166667,0.333333,0.0000,0.083333,0.166667,0.166667,0.083333
375861,375861,1.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
375862,375862,0.000000,1.000000,0.0000,0.000000,0.000000,0.000000,0.000000


**[주말방문비율]** - (1)

In [15]:
f = data.groupby('CLNT_ID')['SESS_DT'].agg([('weekend_visit_prob', lambda x: np.mean(x.dt.dayofweek > 4))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,weekend_visit_prob
0,0,1.000000
1,1,0.125000
2,2,0.000000
3,3,1.000000
4,4,0.333333
...,...,...
375859,375859,0.000000
375860,375860,0.250000
375861,375861,0.000000
375862,375862,0.000000


**[월별방문비율]** - (6)

In [16]:
f = data.groupby('CLNT_ID')['SESS_DT'].agg([('apr_visit_prob', lambda x: np.mean(x.dt.month.isin([4]))),
                                            ('may_visit_prob', lambda x: np.mean(x.dt.month.isin([5]))),
                                            ('jun_visit_prob', lambda x: np.mean(x.dt.month.isin([6]))),
                                            ('jul_visit_prob', lambda x: np.mean(x.dt.month.isin([7]))),
                                            ('aug_visit_prob', lambda x: np.mean(x.dt.month.isin([8]))),
                                            ('sep_visit_prob', lambda x: np.mean(x.dt.month.isin([9])))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,apr_visit_prob,may_visit_prob,jun_visit_prob,jul_visit_prob,aug_visit_prob,sep_visit_prob
0,0,0.00,0.000000,1.000000,0.000000,0.000000,0.000000
1,1,0.00,0.375000,0.625000,0.000000,0.000000,0.000000
2,2,0.00,0.000000,0.909091,0.000000,0.000000,0.090909
3,3,0.00,0.000000,0.000000,0.000000,0.000000,1.000000
4,4,0.00,0.666667,0.000000,0.000000,0.333333,0.000000
...,...,...,...,...,...,...,...
375859,375859,0.00,1.000000,0.000000,0.000000,0.000000,0.000000
375860,375860,0.25,0.333333,0.250000,0.166667,0.000000,0.000000
375861,375861,0.00,0.000000,1.000000,0.000000,0.000000,0.000000
375862,375862,0.00,0.000000,0.000000,1.000000,0.000000,0.000000


**[주방문요일]** - (7)

In [8]:
s = data.groupby('CLNT_ID')['SESS_DT'].agg([('most_visit_day', lambda x : x.dt.day_name().value_counts().idxmax())]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,CLNT_ID,most_visit_day_Friday,most_visit_day_Monday,most_visit_day_Saturday,most_visit_day_Sunday,most_visit_day_Thursday,most_visit_day_Tuesday,most_visit_day_Wednesday
0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,1,0
2,2,0,0,0,0,1,0,0
3,3,0,0,0,1,0,0,0
4,4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,1,0
375860,375860,0,0,0,0,0,1,0
375861,375861,0,1,0,0,0,0,0
375862,375862,0,0,0,0,0,1,0


**[주방문월]** - (6)

In [16]:
s = data.groupby('CLNT_ID')['SESS_DT'].agg([('most_visit_month', lambda x : x.dt.month_name().value_counts().idxmax())]).reset_index()
s = pd.get_dummies(s)
s_list.append(s); display(s)

,CLNT_ID,most_visit_month_April,most_visit_month_August,most_visit_month_July,most_visit_month_June,most_visit_month_May,most_visit_month_September
0,0,0,0,0,1,0,0
1,1,0,0,0,1,0,0
2,2,0,0,0,1,0,0
3,3,0,0,0,0,0,1
4,4,0,0,0,0,1,0
...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,1,0
375860,375860,0,0,0,0,1,0
375861,375861,0,0,0,1,0,0
375862,375862,0,0,1,0,0,0


**[방문주기규칙성]** - (1)

In [26]:
f = data.groupby('CLNT_ID')['SESS_DT'].agg([('sess_dt_unique','unique')]).reset_index()
f['vist_cycle_std'] = f['sess_dt_unique'].apply(lambda x : sess_dt_std(x))
f = f.iloc[:,[0,2]]
f_list.append(f); display(f)

,CLNT_ID,vist_cycle_std
0,0,0.000000
1,1,24.351591
2,2,81.000000
3,3,0.000000
4,4,102.000000
...,...,...
375859,375859,0.000000
375860,375860,54.936327
375861,375861,0.000000
375862,375862,0.000000


**[상품구매주기]** - (1)

In [27]:
f = data.groupby('CLNT_ID')['SESS_DT'].agg([('pd_buy_cycle',lambda x : int((x.max() - x.min()).days)/x.nunique())]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,pd_buy_cycle
0,0,0.000000
1,1,7.250000
2,2,40.500000
3,3,0.000000
4,4,51.000000
...,...,...
375859,375859,0.000000
375860,375860,12.444444
375861,375861,0.000000
375862,375862,0.000000


**[방문추세]** - (1)

In [28]:
week_to = data.SESS_DT.max()
week_trans = []
for i in range(27):
    week_from = week_to + pd.DateOffset(weeks=-1)
    week_trans.append(data.query('@week_from < SESS_DT <= @week_to')
                          .groupby('CLNT_ID')['SESS_ID'].agg([(f'w{27-i}', 'count')]).reset_index())
    week_to = week_from
    
f = pd.DataFrame({'CLNT_ID': data.CLNT_ID.unique()})

for w in week_trans[::-1]:
    f = pd.merge(f, w, how='left')
f = f.fillna(0)

f['visit_trend'] = f.apply(lambda x: np.polyfit(range(27), x[1:], 1)[0].round(3), axis=1)
f = f.iloc[:,[0,-1]]
f_list.append(f); display(f)

,CLNT_ID,visit_trend
0,128408,-0.083
1,28610,-0.127
2,105097,0.075
3,101867,-0.241
4,148630,-0.007
...,...,...
375859,271134,0.010
375860,353125,-0.015
375861,290557,-0.004
375862,372171,-0.017


**[상,하반기 방문건수]** - (2)

In [29]:
data['상하반기'] = data['SESS_DT'].dt.month.apply(lambda x : 1 if x in [4,5,6] else 0)
f = data.groupby('CLNT_ID')['상하반기'].agg([('first_half_visit_cnt', lambda x : x.isin([1]).sum()),
                                             ('second_half_visit_cnt', lambda x : x.isin([0]).sum())]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,first_half_visit_cnt,second_half_visit_cnt
0,0,2,0
1,1,16,0
2,2,10,1
3,3,0,8
4,4,4,2
...,...,...,...
375859,375859,3,0
375860,375860,10,2
375861,375861,2,0
375862,375862,0,4


**[휴면고객여부]** - (1)

In [17]:
s = data.groupby('CLNT_ID')['SESS_DT'].agg([
        ('resting_clnt', lambda x: 1 if x.max() < pd.to_datetime('2018-06-01') else 0)]).reset_index()
s_list.append(s); display(s)

,CLNT_ID,resting_clnt
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
375859,375859,1
375860,375860,0
375861,375861,0
375862,375862,0


**[총페이지수, 평균페이지수, 최소페이지수, 최대페이지수, 페이지수표준편차]** - (5)

In [31]:
f = data.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].agg([('page_view_sum', np.sum),
                                                    ('page_view_mean', np.mean),
                                                    ('page_view_min', np.min),
                                                    ('page_view_max', np.max),
                                                    ('page_view_std', np.std),]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,page_view_sum,page_view_mean,page_view_min,page_view_max,page_view_std
0,0,118.0,59.000000,59.0,59.0,0.000000
1,1,2045.0,127.812500,102.0,161.0,18.655540
2,2,2906.0,264.181818,66.0,284.0,65.729473
3,3,744.0,93.000000,93.0,93.0,0.000000
4,4,340.0,56.666667,54.0,62.0,4.131182
...,...,...,...,...,...,...
375859,375859,399.0,133.000000,133.0,133.0,0.000000
375860,375860,960.0,80.000000,26.0,154.0,42.034618
375861,375861,186.0,93.000000,93.0,93.0,0.000000
375862,375862,340.0,85.000000,85.0,85.0,0.000000


**[총세션시간, 평균세션시간, 최소세션시간, 최대세션시간, 세션시간표준편차]** - (5)

In [32]:
f = data.groupby('CLNT_ID')['TOT_SESS_HR_V'].agg([('sess_hr_sum', np.sum),
                                                  ('sess_hr_mean', np.mean),
                                                  ('sess_hr_min', np.min),
                                                  ('sess_hr_max', np.max),
                                                  ('sess_hr_std', np.std),]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,sess_hr_sum,sess_hr_mean,sess_hr_min,sess_hr_max,sess_hr_std
0,0,1844,922.000000,922,922,0.000000
1,1,22160,1385.000000,620,3063,901.788667
2,2,48241,4385.545455,371,4787,1331.474098
3,3,8408,1051.000000,1051,1051,0.000000
4,4,4198,699.666667,515,792,143.042185
...,...,...,...,...,...,...
375859,375859,3858,1286.000000,1286,1286,0.000000
375860,375860,15852,1321.000000,304,2399,645.931744
375861,375861,3480,1740.000000,1740,1740,0.000000
375862,375862,8584,2146.000000,2146,2146,0.000000


**[사용기기다양성]** - (1)

In [33]:
f = data.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('use_dvc_cnt','nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,use_dvc_cnt
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
375859,375859,1
375860,375860,1
375861,375861,1
375862,375862,1


**[기기별사용비율]** - (3)

In [34]:
f = data.groupby(['CLNT_ID'])['SESS_ID'].agg([('세션수',np.size)]).reset_index()
dvc_cnt = data.groupby('CLNT_ID')['DVC_CTG_NM'].agg([('desktop수', lambda x : np.sum(x.isin(['desktop']))),
                                                      ('mobile수', lambda x : np.sum(x.isin(['mobile']))),
                                                      ('tablet수', lambda x : np.sum(x.isin(['tablet'])))]).reset_index()
f = pd.merge(f, dvc_cnt, how = 'left')
f['desktop_prob'] = f['desktop수']/f['세션수']
f['mobile_prob'] = f['mobile수']/f['세션수']
f['tablet_prob'] = f['tablet수']/f['세션수']

f = f.iloc[:,[0,5,6,7]]
f_list.append(f); display(f)

,CLNT_ID,desktop_prob,mobile_prob,tablet_prob
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,0.0,1.0,0.0
3,3,0.0,1.0,0.0
4,4,0.0,1.0,0.0
...,...,...,...,...
375859,375859,0.0,1.0,0.0
375860,375860,0.0,1.0,0.0
375861,375861,0.0,1.0,0.0
375862,375862,0.0,1.0,0.0


**[지역다양성 - 대,중]** - (2)

In [35]:
f = data.groupby('CLNT_ID')['ZON_NM'].agg([('zon_num','nunique')]).reset_index()
f_list.append(f); display(f)

f = data.groupby('CLNT_ID')['CITY_NM'].agg([('city_num', 'nunique')]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,zon_num
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
375859,375859,1
375860,375860,1
375861,375861,1
375862,375862,1


,CLNT_ID,city_num
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
375859,375859,1
375860,375860,1
375861,375861,1
375862,375862,1


**[지역대분류별 구매건수]** - (16)

In [18]:
s = pd.pivot_table(data, values = 'PD_C', index = 'CLNT_ID', columns = 'ZON_NM', 
                   aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

ZON_NM,CLNT_ID,Busan,Chungcheongbuk-do,Chungcheongnam-do,Daegu,Daejeon,Gangwon-do,Gwangju,Gyeonggi-do,Gyeongsangbuk-do,Gyeongsangnam-do,Incheon,Jeju-do,Jeollabuk-do,Jeollanam-do,Seoul,Ulsan
0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11
3,3,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0
375860,375860,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
375862,375862,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0


**[지역중분류별 구매건수]** - (162)

In [19]:
s = pd.pivot_table(data, values = 'PD_C', index = 'CLNT_ID', columns = 'CITY_NM', 
                   aggfunc = 'count', fill_value = 0).reset_index()
s_list.append(s); display(s)

CITY_NM,CLNT_ID,(not set),Andong,Ansan-si,Anseong,Anyang,Asan-si,Boeun-gun,Bonghwa-gun,Boryeong-si,...,Yeongdeok-gun,Yeongdong-gun,Yeonggwang-gun,Yeongi-gun,Yeongju-si,Yeongwol-gun,Yeongyang-gun,Yeosu-si,Yesan-gun,Yongin-si
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375860,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375862,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 복합 Featrues

**[페이지평균접속시간]** - (1)

In [36]:
hr = data.groupby('CLNT_ID')['TOT_SESS_HR_V'].sum().reset_index()
page = data.groupby('CLNT_ID')['TOT_PAG_VIEW_CT'].sum().reset_index()
f = pd.merge(hr, page, how = 'left')
f['page_hr_mean'] = f['TOT_SESS_HR_V']/f['TOT_PAG_VIEW_CT']

f = f.iloc[:,[0,3]]
f_list.append(f); display(f)

,CLNT_ID,page_hr_mean
0,0,15.627119
1,1,10.836186
2,2,16.600482
3,3,11.301075
4,4,12.347059
...,...,...
375859,375859,9.669173
375860,375860,16.512500
375861,375861,18.709677
375862,375862,25.247059


**[검색대비구매율]** - (1)

In [37]:
search_buy = data.groupby('CLNT_ID').agg({'PD_BUY_CT' : np.sum,'SEARCH_CNT': np.sum}).reset_index()
search_buy['search_buy_prob'] = search_buy['PD_BUY_CT']/search_buy['SEARCH_CNT']
f = search_buy[['CLNT_ID','search_buy_prob']]
f_list.append(f); display(f)

,CLNT_ID,search_buy_prob
0,0,1.000000
1,1,0.695652
2,2,0.521739
3,3,0.666667
4,4,0.545455
...,...,...
375859,375859,0.333333
375860,375860,0.521739
375861,375861,0.666667
375862,375862,1.000000


**[총재구매건수, 평균재구매건수]** - (2)

In [38]:
f = data.groupby(['CLNT_ID','CLAC1_NM'])['SESS_DT'].agg([
        ('visit_cnt','nunique')]).reset_index().groupby('CLNT_ID')['visit_cnt'].agg([
        ('rebuy_mean',lambda x : (sum(x) - len(x))/len(x)),
        ('rebuy_sum', lambda x : sum(x) - len(x))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,rebuy_mean,rebuy_sum
0,0,0.000000,0
1,1,0.666667,2
2,2,0.000000,0
3,3,0.000000,0
4,4,0.000000,0
...,...,...,...
375859,375859,0.000000,0
375860,375860,8.000000,8
375861,375861,0.000000,0
375862,375862,0.000000,0


**[세션방문당구매액]** - (1)

In [39]:
sess_visit = data.groupby('CLNT_ID')['SESS_ID'].agg(lambda x : x.nunique())
f = (data.groupby('CLNT_ID')['AMOUNT'].sum() / sess_visit).reset_index().rename(columns = {0 : 'sess_vist_amount_sum'})
f_list.append(f); display(f)

,CLNT_ID,sess_vist_amount_sum
0,0,86500.000000
1,1,255200.000000
2,2,280000.000000
3,3,851200.000000
4,4,62600.000000
...,...,...
375859,375859,80000.000000
375860,375860,100311.111111
375861,375861,91800.000000
375862,375862,92000.000000


**[세션방문당구매건수]** - (1)

In [40]:
sess_visit = data.groupby('CLNT_ID')['SESS_ID'].agg(lambda x : x.nunique())
f = (data.groupby('CLNT_ID')['AMOUNT'].size() / sess_visit).reset_index().rename(columns = {0 : 'sess_vist_buy_num'})
f_list.append(f); display(f)

,CLNT_ID,sess_vist_buy_num
0,0,2.000000
1,1,3.200000
2,2,5.500000
3,3,8.000000
4,4,3.000000
...,...,...
375859,375859,3.000000
375860,375860,1.333333
375861,375861,2.000000
375862,375862,4.000000


**[최근1달간구매금액, 최근1달간구매건수]** - (2)

In [43]:
start = str(pd.to_datetime(data.SESS_DT.max()) - pd.offsets.MonthBegin(1))
f = data.query('SESS_DT >= @start').groupby('CLNT_ID')['AMOUNT'].agg([('recent_one_month_amount_sum', np.sum),
                                                                       ('recent_one_month_buy_num', np.size)]).reset_index()
clnt_id = pd.DataFrame({'CLNT_ID': data.CLNT_ID.unique()})
clnt_id = clnt_id.sort_values(by = 'CLNT_ID')
f = pd.merge(clnt_id, f, how = 'left').fillna(0)
f_list.append(f); display(f)

,CLNT_ID,recent_one_month_amount_sum,recent_one_month_buy_num
0,0,0.0,0.0
1,1,0.0,0.0
2,2,20000.0,1.0
3,3,851200.0,8.0
4,4,0.0,0.0
...,...,...,...
375859,375859,0.0,0.0
375860,375860,0.0,0.0
375861,375861,0.0,0.0
375862,375862,0.0,0.0


**[베스트셀러구매비율, 구매건수]** - (2)

In [48]:
best_seller = list(data.CLAC1_NM.value_counts().index[:10])

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)]).reset_index()
bsbn = data.query('CLAC1_NM == @best_seller').groupby('CLNT_ID')['AMOUNT'].agg([('best_seller_buy_num', np.size)]).reset_index()

f = pd.merge(f, bsbn, on = 'CLNT_ID', how = 'left').fillna(0)
f['best_seller_buy_prob'] = f['best_seller_buy_num'] / f['구매건수']
f = f.iloc[:,[0,2,3]]
f_list.append(f); display(f)

,CLNT_ID,best_seller_buy_num,best_seller_buy_prob
0,0,0.0,0.000
1,1,14.0,0.875
2,2,11.0,1.000
3,3,8.0,1.000
4,4,6.0,1.000
...,...,...,...
375859,375859,3.0,1.000
375860,375860,12.0,1.000
375861,375861,2.0,1.000
375862,375862,4.0,1.000


**[평균대비구매]** - (1)

In [49]:
a_mean = data.groupby('CLNT_ID')['AMOUNT'].mean().mean()
f = data.groupby('CLNT_ID')['AMOUNT'].agg([('mean_cont_amount',lambda x : sum(x) / a_mean)]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,mean_cont_amount
0,0,1.430542
1,1,21.102563
2,2,9.261313
3,3,14.077196
4,4,2.070565
...,...,...
375859,375859,1.323045
375860,375860,14.930560
375861,375861,1.518194
375862,375862,1.521501


**[fm스코어]** - (1)

In [50]:
f = data.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)]).reset_index() 
f['총구매액'] = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum)]).reset_index().총구매액

f['f'] = pd.qcut(f['구매건수'], q = 4, labels = range(1, 5)).astype(int)
f['m'] = pd.qcut(f['총구매액'], q = 4, labels = range(1, 5)).astype(int)
f['fm_score'] =  (3 * f['f']) + (2 * f['m'])

f = f.iloc[:,[0,5]]
f_list.append(f); display(f)

,CLNT_ID,fm_score
0,0,7
1,1,20
2,2,17
3,3,17
4,4,13
...,...,...
375859,375859,10
375860,375860,17
375861,375861,7
375862,375862,10


**[식료품 총구매액, 구매비율]** - (2)

In [51]:
clac1_food = ['과일', '음료', '건강식품', '냉장식품', '냉동식품', '축산물']

f = data.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액','sum')]).reset_index()
a_food = data.query('CLAC1_NM == @clac1_food').groupby('CLNT_ID')['AMOUNT'].agg([('amount_clac1_food','sum')]).reset_index()
f = pd.merge(f, a_food, how='left').fillna(0)
f['amount_clac1_food_prob'] = f['amount_clac1_food'] / f['총구매액']

f = f.iloc[:,[0,2,3]]
f_list.append(f); display(f)

,CLNT_ID,amount_clac1_food,amount_clac1_food_prob
0,0,0.0,0.00000
1,1,192000.0,0.15047
2,2,0.0,0.00000
3,3,0.0,0.00000
4,4,0.0,0.00000
...,...,...,...
375859,375859,0.0,0.00000
375860,375860,0.0,0.00000
375861,375861,0.0,0.00000
375862,375862,0.0,0.00000


**[방문일별구매액분산]** - (1)

In [52]:
f = data.groupby(['CLNT_ID','SESS_DT'])['AMOUNT'].sum().reset_index().groupby('CLNT_ID')['AMOUNT'].agg([
                                                                ('sess_dt_amount_var',lambda x : np.var(x))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,sess_dt_amount_var
0,0,0.000000e+00
1,1,7.363500e+09
2,2,6.760000e+10
3,3,0.000000e+00
4,4,1.299600e+08
...,...,...
375859,375859,0.000000e+00
375860,375860,7.083894e+09
375861,375861,0.000000e+00
375862,375862,0.000000e+00


**[단독상품 구매비율, 구매건수]** - (2)

In [53]:
f = data.groupby('CLNT_ID')['AMOUNT'].agg([('구매건수', np.size)])
f2 = data.groupby(['CLNT_ID','CLAC2_NM'])['AMOUNT'].agg([
            ('구매건수', np.size)]).reset_index().query('구매건수 == 1').groupby('CLNT_ID')['구매건수'].agg([
            ('ond_pd_buy_num', 'count')]).reset_index()

f = pd.merge(f,f2, on = 'CLNT_ID', how='left').fillna(0)
f['one_pd_buy_prob'] = f['ond_pd_buy_num']/f['구매건수']
f = f.iloc[:,[0,2,3]]

f_list.append(f); display(f)

,CLNT_ID,ond_pd_buy_num,one_pd_buy_prob
0,0,2.0,1.000000
1,1,1.0,0.062500
2,2,1.0,0.090909
3,3,0.0,0.000000
4,4,0.0,0.000000
...,...,...,...
375859,375859,0.0,0.000000
375860,375860,0.0,0.000000
375861,375861,0.0,0.000000
375862,375862,0.0,0.000000


**[방문일당최대구매액평균]** - (1)

In [54]:
f = data.groupby(['CLNT_ID','SESS_DT'])['AMOUNT'].agg([('일별최대구매액',np.max)]).groupby('CLNT_ID')['일별최대구매액'].agg([
            ('sess_dt_amount_max_mean', lambda x : np.mean(x))]).reset_index()
f_list.append(f); display(f)

,CLNT_ID,sess_dt_amount_max_mean
0,0,81000.0
1,1,96500.0
2,2,41050.0
3,3,110400.0
4,4,24900.0
...,...,...
375859,375859,35000.0
375860,375860,66200.0
375861,375861,45900.0
375862,375862,23000.0


**[추석기간총구매액, 구매건수]** - (2)

In [60]:
chuseok = pd.to_datetime([20180912, 20180913, 20180914, 20180915, 20180916, 20180917, 20180918, 20180919], format='%Y%m%d')
f = data.query('SESS_DT in @chuseok').groupby('CLNT_ID')['AMOUNT'].agg([('chuseok_amount_sum','sum'),
                                                                        ('chuseok_buy_num','count')]).reset_index()
clnt_id = pd.DataFrame({'CLNT_ID': data.CLNT_ID.unique()})
clnt_id = clnt_id.sort_values(by = 'CLNT_ID')

f = pd.merge(clnt_id, f, how = 'left').fillna(0)
f_list.append(f); display(f)

,CLNT_ID,chuseok_amount_sum,chuseok_buy_num
0,0,0.0,0.0
1,1,0.0,0.0
2,2,0.0,0.0
3,3,0.0,0.0
4,4,0.0,0.0
...,...,...,...
375859,375859,0.0,0.0
375860,375860,0.0,0.0
375861,375861,0.0,0.0
375862,375862,0.0,0.0


# Feature Merge

### Numeric Feature Merge

In [65]:
features = fmerge(f_list, features); features

,CLNT_ID,amount_sum,buy_num,amount_mean,amount_max,amount_min,amount_std,amount_cv,visit_act_div,most_visit_act,...,fm_score,amount_clac1_food,amount_clac1_food_prob,sess_dt_amount_var,ond_pd_buy_num,one_pd_buy_prob,sess_dt_amount_max_mean,chuseok_amount_sum,chuseok_buy_num,most_search_kwd_cnt
0,0,86500,2,43250.000000,81000,5500,53386.561980,0.872832,1,63,...,7,0.0,0.00000,0.000000e+00,2.0,1.000000,81000.0,0.0,0.0,2
1,1,1276000,16,79750.000000,99000,39000,22944.861444,0.278574,6,114,...,20,192000.0,0.15047,7.363500e+09,1.0,0.062500,96500.0,0.0,0.0,6
2,2,560000,11,50909.090909,62100,20000,13065.255799,0.244696,2,250,...,17,0.0,0.00000,6.760000e+10,1.0,0.090909,41050.0,0.0,0.0,6
3,3,851200,8,106400.000000,110400,102400,4276.179871,0.037594,2,21,...,17,0.0,0.00000,0.000000e+00,0.0,0.000000,110400.0,0.0,0.0,6
4,4,125200,6,20866.666667,37000,12800,12496.826264,0.546709,2,17,...,13,0.0,0.00000,1.299600e+08,0.0,0.000000,24900.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,80000,3,26666.666667,35000,20000,7637.626158,0.233854,2,109,...,10,0.0,0.00000,0.000000e+00,0.0,0.000000,35000.0,0.0,0.0,9
375860,375860,902800,12,75233.333333,149000,35400,38881.248889,0.494807,9,38,...,17,0.0,0.00000,7.083894e+09,0.0,0.000000,66200.0,0.0,0.0,14
375861,375861,91800,2,45900.000000,45900,45900,0.000000,0.000000,1,46,...,7,0.0,0.00000,0.000000e+00,0.0,0.000000,45900.0,0.0,0.0,2
375862,375862,92000,4,23000.000000,23000,23000,0.000000,0.000000,1,77,...,10,0.0,0.00000,0.000000e+00,0.0,0.000000,23000.0,0.0,0.0,2


In [6]:
features.to_csv('features.csv', encoding = 'UTF-8', index = False)

### Categorical & Pivot Feature Merge

In [17]:
sparse = fmerge(s_list, sparse); sparse

,CLNT_ID,1,2,3,4,5,6,7,8,9,...,most_visit_act_90,most_visit_act_91,most_visit_act_92,most_visit_act_93,most_visit_act_94,most_visit_act_95,most_visit_act_96,most_visit_act_97,most_visit_act_98,most_visit_act_99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375859,375859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375860,375860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375861,375861,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
375862,375862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
sparse.to_csv('sparse_features.csv', encoding = 'UTF-8', index = False)

NameError: name 'sparse' is not defined

In [4]:
s = pd.read_csv('sparse_pca.csv', encoding = 'UTF-8')